A forecasting model is to be built based on simulated user information and past loan history, for the amount of loan a user may apply for next month

In [1]:
import pandas as pd

# display all varibales
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
user = pd.read_csv('user.csv')
loan = pd.read_csv('loan.csv')
user.head()
loan.head()

,uid,age,sex,active_date,limit
0,26308,30,1,2016-02-16,15000.0
1,78209,40,1,2016-02-21,5000.0
2,51930,35,1,2016-04-19,25000.0
3,10113,25,1,2016-03-12,25000.0
4,17067,35,1,2016-02-16,15000.0


,uid,loan_time,loan_amount,plannum
0,12135,2016-08-03 00:05:26,500.0,1
1,41403,2016-08-03 00:13:25,8000.0,3
2,74458,2016-08-03 00:13:58,2000.0,1
3,12959,2016-08-03 00:19:33,500.0,1
4,89641,2016-08-03 00:23:13,1000.0,1


In [3]:
user['active_month'] = user.active_date.apply(lambda x: x[0:7].replace('-', '')).astype(int)
user['active_date'] = pd.to_datetime(user.active_date)
loan['date'] = pd.to_datetime(loan.loan_time.apply(lambda x: x[0:10]))
loan['month'] = loan.loan_time.apply(lambda x: x[0:7].replace('-', '')).astype(int)

In [4]:
loan_stat_by_month = loan.groupby(['uid', 'month']).agg({'loan_amount': ['sum', 'count'],'plannum':['sum']})\
    .reset_index()
loan_stat_by_month.head()
loan_stat_by_month.columns = ['uid', 'month', 'loan_sum', 'loan_count','plannum_sum']
loan_stat_by_month.head()

# build train data
train = user.merge(loan_stat_by_month[loan_stat_by_month.month == 201610][['uid', 'loan_sum']], 
                   on='uid', how='left')
train = train.rename(columns={'loan_sum': 'target'})
train.target = train.target.fillna(0)
train = train.merge(loan_stat_by_month[loan_stat_by_month.month == 201609]\
                    [['uid', 'loan_sum', 'loan_count','plannum_sum']],
                    on='uid', how='left')
train = train.rename(columns={'loan_sum': 'prev_1_loan_sum', 'loan_count': 'prev_1_loan_count'})
train = train.fillna(0)
train.head()

test = user.merge(loan_stat_by_month[loan_stat_by_month.month == 201611][['uid', 'loan_sum']], 
                   on='uid', how='left')
test = test.rename(columns={'loan_sum': 'target'})
test.target = test.target.fillna(0)
test = test.merge(loan_stat_by_month[loan_stat_by_month.month == 201610]\
                    [['uid', 'loan_sum', 'loan_count','plannum_sum']],
                    on='uid', how='left')
test = test.rename(columns={'loan_sum': 'prev_1_loan_sum', 'loan_count': 'prev_1_loan_count'})
test = test.fillna(0)
test.head()

uid   month loan_amount       plannum
                      sum count     sum
0   4  201608      9000.0     2       2
1   4  201610     16000.0     2      13
2   4  201611      2000.0     1       3
3   5  201611     40000.0     1      12
4   6  201609     24000.0     1       3

,uid,month,loan_sum,loan_count,plannum_sum
0,4,201608,9000.0,2,2
1,4,201610,16000.0,2,13
2,4,201611,2000.0,1,3
3,5,201611,40000.0,1,12
4,6,201609,24000.0,1,3


,uid,age,sex,active_date,limit,active_month,target,prev_1_loan_sum,prev_1_loan_count,plannum_sum
0,26308,30,1,2016-02-16,15000.0,201602,16500.0,3500.0,2.0,2.0
1,78209,40,1,2016-02-21,5000.0,201602,0.0,0.0,0.0,0.0
2,51930,35,1,2016-04-19,25000.0,201604,0.0,30000.0,1.0,6.0
3,10113,25,1,2016-03-12,25000.0,201603,0.0,0.0,0.0,0.0
4,17067,35,1,2016-02-16,15000.0,201602,0.0,0.0,0.0,0.0


,uid,age,sex,active_date,limit,active_month,target,prev_1_loan_sum,prev_1_loan_count,plannum_sum
0,26308,30,1,2016-02-16,15000.0,201602,7000.0,16500.0,8.0,12.0
1,78209,40,1,2016-02-21,5000.0,201602,0.0,0.0,0.0,0.0
2,51930,35,1,2016-04-19,25000.0,201604,0.0,0.0,0.0,0.0
3,10113,25,1,2016-03-12,25000.0,201603,0.0,0.0,0.0,0.0
4,17067,35,1,2016-02-16,15000.0,201602,0.0,0.0,0.0,0.0


In [5]:
# At least need to repay
train['need_repay'] = train.prev_1_loan_sum / train.plannum_sum
train = train.fillna(0)
test['need_repay'] = test.prev_1_loan_sum / test.plannum_sum
test = test.fillna(0)
train.head()
test.head()

,uid,age,sex,active_date,limit,active_month,target,prev_1_loan_sum,prev_1_loan_count,plannum_sum,need_repay
0,26308,30,1,2016-02-16,15000.0,201602,16500.0,3500.0,2.0,2.0,1750.0
1,78209,40,1,2016-02-21,5000.0,201602,0.0,0.0,0.0,0.0,0.0
2,51930,35,1,2016-04-19,25000.0,201604,0.0,30000.0,1.0,6.0,5000.0
3,10113,25,1,2016-03-12,25000.0,201603,0.0,0.0,0.0,0.0,0.0
4,17067,35,1,2016-02-16,15000.0,201602,0.0,0.0,0.0,0.0,0.0


,uid,age,sex,active_date,limit,active_month,target,prev_1_loan_sum,prev_1_loan_count,plannum_sum,need_repay
0,26308,30,1,2016-02-16,15000.0,201602,7000.0,16500.0,8.0,12.0,1375.0
1,78209,40,1,2016-02-21,5000.0,201602,0.0,0.0,0.0,0.0,0.0
2,51930,35,1,2016-04-19,25000.0,201604,0.0,0.0,0.0,0.0,0.0
3,10113,25,1,2016-03-12,25000.0,201603,0.0,0.0,0.0,0.0,0.0
4,17067,35,1,2016-02-16,15000.0,201602,0.0,0.0,0.0,0.0,0.0


In [6]:
X_train = train[['age', 'sex', 'limit', 'prev_1_loan_sum', 'prev_1_loan_count','need_repay']]
y_train = train.target
ids_train = train.uid

X_test = test[['age', 'sex', 'limit', 'prev_1_loan_sum', 'prev_1_loan_count','need_repay']]
y_test = test.target
ids_test = test.uid

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# Random Forest
random_forest = RandomForestRegressor(n_estimators=20)

random_forest.fit(X_train, y_train)

preds = random_forest.predict(X_test)

print("RMSE score {}".format(mean_squared_error(preds, y_test) ** 0.5))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

RMSE score 8362.413594681104


In [8]:
submission = pd.DataFrame({
    'uid': ids_test,
    'loan_sum': preds
})
submission.to_csv('MICHAEL GONG.csv', index=False)